In [1]:
import numpy as np
import re

In [2]:
with open("05-input", "r") as file:
    lines = file.readlines()
data = [line.replace("\n", "") for line in lines]
data = "\n".join(data)
data[:10]

'seeds: 515'

In [3]:
test_data = """seeds: 79 14 55 13

seed-to-soil map:
50 98 2
52 50 48

soil-to-fertilizer map:
0 15 37
37 52 2
39 0 15

fertilizer-to-water map:
49 53 8
0 11 42
42 0 7
57 7 4

water-to-light map:
88 18 7
18 25 70

light-to-temperature map:
45 77 23
81 45 19
68 64 13

temperature-to-humidity map:
0 69 1
1 0 69

humidity-to-location map:
60 56 37
56 93 4"""

def preprocess_data (data):
    output = [line.split(":") for line in data.split("\n\n")]
    output = {line[0].replace(" map", ""): line[1].split("\n") for line in output}
    output = {a: [item.strip() for item in b if item] for a,b in output.items()}
    return output

test_data = preprocess_data(test_data)

In [4]:
data = preprocess_data(data)

In [5]:
test_data

{'seeds': ['79 14 55 13'],
 'seed-to-soil': ['50 98 2', '52 50 48'],
 'soil-to-fertilizer': ['0 15 37', '37 52 2', '39 0 15'],
 'fertilizer-to-water': ['49 53 8', '0 11 42', '42 0 7', '57 7 4'],
 'water-to-light': ['88 18 7', '18 25 70'],
 'light-to-temperature': ['45 77 23', '81 45 19', '68 64 13'],
 'temperature-to-humidity': ['0 69 1', '1 0 69'],
 'humidity-to-location': ['60 56 37', '56 93 4']}

In [6]:
def solution (data):
    seeds = [int(seed) for seed in data["seeds"][0].split(" ")]
    map_names = ['seed-to-soil', 'soil-to-fertilizer', 'fertilizer-to-water', 'water-to-light', 'light-to-temperature', 'temperature-to-humidity', 'humidity-to-location']
    map_ranges = {}
    for map_name in map_names:
        map_ranges[map_name] = {"source": [], "dest": []}
        for map_range_str in data[map_name]:
            parse_map_range = [int(num) for num in map_range_str.split(" ")]
            map_ranges[map_name]["source"] = map_ranges[map_name]["source"] + [(parse_map_range[1], parse_map_range[1] + (parse_map_range[2] - 1))]
            map_ranges[map_name]["dest"] = map_ranges[map_name]["dest"] + [(parse_map_range[0], parse_map_range[0] + (parse_map_range[2] - 1))]
    
    seed_locations = {}
    for seed in seeds:

        source = seed
        for map_name in map_names:
            dest = source
            for index, (start, end) in enumerate(map_ranges[map_name]["source"]):
                # print(start, end)
                if source >= start and source <= end:
                    delta = source - start
                    dest = map_ranges[map_name]["dest"][index][0] + delta

            # print(map_name, "source", source, "dest", dest)
            source = dest

        seed_locations[seed] = dest


    return seed_locations, map_ranges


locs, ranges = solution(data)
# locs, ranges = solution(test_data)
# display(ranges)
display(locs)
print(min(locs.values()))

{515785082: 4202284817,
 87905039: 2462188886,
 2104518691: 4287508490,
 503149843: 4050220846,
 720333403: 2625685932,
 385234193: 3507348822,
 1357904101: 4074800133,
 283386167: 892121398,
 93533455: 2467817302,
 128569683: 1104301931,
 2844655470: 3337138858,
 24994629: 1860608055,
 3934515023: 2980390369,
 67327818: 2441611665,
 2655687716: 2734303452,
 8403417: 457535844,
 3120497449: 599623118,
 107756881: 1083489129,
 4055128129: 2353465194,
 9498708: 458631135}

457535844


In [29]:
def solution (data, verbose=False):
    seeds = [int(seed) for seed in data["seeds"][0].split(" ")]
    map_names = ['seed-to-soil', 'soil-to-fertilizer', 'fertilizer-to-water', 'water-to-light', 'light-to-temperature', 'temperature-to-humidity', 'humidity-to-location']
    map_ranges = {}
    for map_name in map_names:
        map_ranges[map_name] = {"source": [], "dest": []}
        for map_range_str in data[map_name]:
            parse_map_range = [int(num) for num in map_range_str.split(" ")]
            map_ranges[map_name]["source"] = map_ranges[map_name]["source"] + [(parse_map_range[1], parse_map_range[1] + (parse_map_range[2] - 1))]
            map_ranges[map_name]["dest"] = map_ranges[map_name]["dest"] + [(parse_map_range[0], parse_map_range[0] + (parse_map_range[2] - 1))]
    
    all_seeds = []
    for seed_start, seed_num in zip(seeds[0::2], seeds[1::2]):
        next_seeds = [(seed_start, seed_start + (seed_num - 1))]
        all_seeds = all_seeds + next_seeds

    seed_locations = {}
    for seed_start, seed_end in all_seeds:
        source = [(seed_start, seed_end)]
        for map_name in map_names:
            if verbose:
                print("Starting stage", map_name, "for source", source)

            mapped_ranges = []
            not_mapped_ranges = source
            for index, (start, end) in enumerate(map_ranges[map_name]["source"]):
                if verbose:
                    print("Map range", start, end)
                    print("Source Ranges", not_mapped_ranges)
                not_mapped_ranges_tmp = []
                for src_start, src_end in not_mapped_ranges:

                    if src_start > end or src_end < start:
                        not_mapped_ranges_tmp += [(src_start, src_end)]
                    elif src_start >= start and src_end <= end:
                        delta_start = src_start - start
                        dest_start = map_ranges[map_name]["dest"][index][0] + delta_start
                        delta_end = end - src_end
                        dest_end = map_ranges[map_name]["dest"][index][1] - delta_end
                        mapped_ranges += [(dest_start, dest_end)]
    
                    elif src_start <= start and src_end >= end:
                        if src_start < start:
                            not_mapped_ranges_tmp += [(src_start, start - 1)]
                        if src_end > end:
                            not_mapped_ranges_tmp += [(end+1, src_end)]
                        mapped_ranges += [tuple(map_ranges[map_name]["dest"][index])]

                    elif src_start < start and src_end < end:
                        not_mapped_ranges_tmp += [(src_start, start-1)]
                        dest_start = map_ranges[map_name]["dest"][index][0]
                        delta_end = end - src_end
                        dest_end = map_ranges[map_name]["dest"][index][1] - delta_end
                        mapped_ranges += [(dest_start, dest_end)]

                    elif src_start > start and src_end > end:
                        not_mapped_ranges_tmp += [(end+1, src_end)]
                        delta_start = src_start - start
                        dest_start = map_ranges[map_name]["dest"][index][0] + delta_start
                        dest_end = map_ranges[map_name]["dest"][index][1]
                        mapped_ranges += [(dest_start, dest_end)]

                not_mapped_ranges = not_mapped_ranges_tmp
                if verbose:
                    print("Destination Ranges", mapped_ranges)
                    print("Not yet map ranges", not_mapped_ranges)

            # print(map_name, "source", source, "dest", dest)
            source = mapped_ranges + not_mapped_ranges
            if verbose:
                print("Mapping of stage", map_name, " with destination", source, "\n")

        seed_locations[(seed_start, seed_end)] = source

    return seed_locations, map_ranges


locs, ranges = solution(data, verbose=False)
# locs, ranges = solution(test_data, verbose=True)
# display(ranges)
# display(locs)
sol = min([loc[0] for seeds_loc in locs.values() for loc in seeds_loc])
print(sol)

41222968
